# 总结Youtube视频

最近因为学习AGI，要看很多视频，一个视频10~60分钟不等的时长，很容易诱发拖延症。
意识到总结是AI的传统艺能，所以实现一个ChatGPT总结YouTube视频功能。

In [ ]:
pip install langchain youtube-transcript-api openai promptlayer tiktoken

## 获取字幕

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_id):
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['zh-Hans', 'zh', 'zh-Hans', 'en'])
    last = transcript[-1]['start']
    need_hour = last > 3600
    lines = []

    t = transcript

    for t in transcript:
        h = int(t['start'] // 3600)
        m = int((t['start'] - h*3600) // 60)
        s = int(t['start'] - h*3600 - m*60)
        if need_hour:
            txt = f"[{h:02}:{m:02}:{s:02}] {t['text']}"
        else:
            txt = f"[{m:02}:{s:02}] {t['text']}"
        lines.append(txt)

    return lines

import json
def dump_transcript(video_id):
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['zh-Hans', 'zh', 'zh-Hans', 'en'])
    with open(f'./dataset/{video_id}.json', 'w') as f:
        json.dump(transcript, f)

dump_transcript('aywZrzNaKjs')
lines = get_transcript('aywZrzNaKjs')
transcript = ''.join(lines)

In [ ]:
import tiktoken
token_encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
encoded = token_encoder.encode(transcript)
print(len(transcript), len(encoded))

## 要求ChatGPT总结(v1)

In [1]:
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.chat_models import PromptLayerChatOpenAI

chat = PromptLayerChatOpenAI(pl_tags=["note-youtube-summarize"], temperature=0, verbose=True)

In [ ]:
t = f'''请帮我总结以下内容:
{transcript}
'''

r = chat([HumanMessage(content=t)])

print(r)

## 优化Prompt(v2)

In [ ]:
chunk_prompt = f'''你是一个专业的视频编辑，负责总结视频的精华内容。请以一段话总结整个视频为开始，然后总结视频不同的片段。输出的格式：

视频的一段话总结
[00:00] 片段总结
[00:22] 片段总结
...

把片段控制在20个以内，请保证所有的内容简介、清晰并且完整。

以下是需要总结的视频脚本：
{transcript}
'''

r = chat([HumanMessage(content=chunk_prompt)])

print(r)

In [ ]:
print(lines[-8:])
print(r.content)

## 优化Prompt(v3)
GPT的输入和输出共用一个Token空间，现在有一个新问题，就是当字幕过多时，会导致Token超出上限或者挤压输出的空间。上面的输出到四分半的位置就戛然而止。
解决方案就是把原文拆成多份，然后再处理，处理方法有两种：
1. Refine - 每次把前一份的结果连同当前份的数据一同提交
2. Map-Reduce - 所有的数据处理完成之后，最后统一处理结果数据

Map Reduce的最大优势，是可以并发处理，整体时间会短一些。

In [7]:
import tiktoken

def num_tokens_from_string(string: str, model: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

OVERLAP_LINES = 5
def to_chunks(lines, model_name='gpt-3.5-turbo'):
    n = '\n'.join(lines)
    if num_tokens_from_string(n, model_name) < 3500:
        return [n]

    l = int(len(lines)//2)
    return to_chunks(lines[:l], model_name) + to_chunks(lines[l-OVERLAP_LINES:], model_name)

chunks = to_chunks(lines)
print(f"Chunks: {len(chunks)}", [num_tokens_from_string(c, 'gpt-3.5-turbo') for c in chunks])

Chunks: 2 [2016, 2079]
[00:00] blank chain what is it why should you
[00:03] use it and how does it work let's have a
[00:05] look
[00:07] Lang chain is an open source framework
[00:09] that allows developers working with AI
[00:11] to combine large language models like
[00:14] gbt4 with external sources of
[00:17] computation and data the framework is
[00:20] currently offered as a python or a
[00:22] JavaScript package typescript to be
[00:24] specific in this video we're going to
[00:26] start unpacking the python framework and
[00:29] we're going to see why the popularity of
[00:31] the framework is exploding right now
[00:32] especially after the introduction of
[00:34] gpt4 in March 2023 to understand what
[00:38] need Lang chain fills let's have a look
[00:40] at a practical example so by now we all
[00:43] know that chat typically or tpt4 has an
[00:45] impressive general knowledge we can ask
[00:47] it about almost anything and we'll get a
[00:50] pretty good answer
[00:51] su

In [19]:
from langchain.prompts import PromptTemplate

chunk_template = '''你是一个专业的视频编辑，负责总结视频的精华内容。请以一段话总结整个视频为开始，然后总结视频不同的片段。输出的格式：

视频的一段话总结
[mm:ss] 片段总结
[mm:ss] 片段总结
...

把片段控制在20个以内，不用重复脚本的内容，请保证所有的内容简介、清晰并且完整。

以下是需要总结的视频脚本：
{transcript}
'''
map_prompt = PromptTemplate(template=chunk_template, input_variables=['transcript'])

summeries = [chat([HumanMessage(content=map_prompt.format(transcript=c))]) for c in chunks]

In [26]:
summary_template = """你是一个专业的视频总编辑，负责将其它视频编辑提交的不同视频片段的总结合并成一个总结。你需要以如下的格式输出最终结果：

视频的一段话总结
[mm:ss] 片段总结
[mm:ss] 片段总结

如果其它编辑提交的内容不是中文，请翻译成中文。把片段控制在20个以内，请保证所有的内容简介、清晰并且完整。

以下是需要总结的视频脚本列表：
{transcript}
"""
reduce_prompt = PromptTemplate(template=summary_template, input_variables=['transcript'])

txt = "\n===\n".join(s.content for s in summeries)
res = chat([HumanMessage(content=reduce_prompt.format(transcript=txt))])


In [27]:
# print(reduce_prompt.format(transcript=txt))
print(res.content)
# print(map_prompt.format(transcript=chunks[0]), summeries[0].content)
# print(map_prompt.format(transcript=chunks[1]), summeries[1].content)


视频的一段话总结：本视频介绍了LangChain的核心概念，包括语言模型、聊天模型、提示模板、链、嵌入和向量存储以及代理。同时，还介绍了DaVinci 003完成模型和大型语言模型的概念。

[00:07] Lang chain是一个开源框架，允许开发人员将大型语言模型与外部计算和数据源结合起来，从而构建数据感知和可执行的语言模型应用程序。
[05:50] 介绍了DaVinci 003完成模型和大型语言模型的概念。
[06:02] 介绍了聊天模型GBT 3.5和GBT 4以及如何使用链接链与其交互。
[06:58] 介绍了提示模板的概念以及如何使用它们动态更改提示。
[08:06] 介绍了链的概念以及如何将多个模型组合在一起。
[09:01] 介绍了嵌入和向量存储的概念以及如何将文本转换为嵌入向量并将其存储在Pinecone中。
[11:24] 介绍了代理的概念以及如何使用LangChain执行Python代码。


## Next
测试过程中，出现了不太令人满意的结果，不过这就属于Prompt Engineering的范畴了。